In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

In [2]:
# Load your data
df = pd.read_csv("GSE10950_gene_expression_renamed.csv")
df.head()

,geo_accession,target,EEF1A1,TUBB,TXN,ACTB,SLC35E2,PDCD1LG2,RPS28,IPO13,...,SMCR7.1,NUP107,FTSJ2.1,MGC9712,TRPM3.1,SMAD7,PCYOX1L,EEF1A1.2,ACTB.2,GAPDH.1
0,GSM277495,0,56190.14,1971.4430,16093.71,54993.97,40.00085,5.684647,31040.54,223.2116,...,21.159680,419.1906,173.0200,111.49560,8.302612,624.81980,200.0302,39215.89,28558.14,11303.50
1,GSM277496,1,52176.93,606.7773,18485.05,46674.46,35.96349,18.281510,30282.59,221.1387,...,13.909390,1017.3650,293.4511,51.37997,-7.676422,89.04105,297.8348,48556.75,26478.78,21009.94
2,GSM277497,0,50483.09,8916.9360,10451.59,48284.97,31.27827,13.904390,26012.85,431.4342,...,9.122181,619.0782,139.2744,47.95465,4.253619,331.57000,160.6262,42814.16,42527.69,14241.25
3,GSM277498,1,38690.04,3148.3370,23610.13,36091.75,18.39922,1.182301,25750.38,178.7281,...,-3.768425,1847.4700,354.8188,67.30951,-4.145857,222.88310,215.5376,37049.47,33330.46,32106.39
4,GSM277499,0,54803.84,3906.8750,11740.43,62380.23,59.56140,18.286430,28069.13,376.0385,...,30.748280,462.7539,180.9093,372.56140,-9.525824,307.97920,167.7340,38649.68,36737.00,11845.22


In [3]:
# Separate groups based on 'target' (0 = control, 1 = case)
group_0 = df[df['target'] == 0].drop(columns=['geo_accession', 'target'])
group_1 = df[df['target'] == 1].drop(columns=['geo_accession', 'target'])

In [4]:
group_0.head()

,EEF1A1,TUBB,TXN,ACTB,SLC35E2,PDCD1LG2,RPS28,IPO13,SYT14,AFAP1,...,SMCR7.1,NUP107,FTSJ2.1,MGC9712,TRPM3.1,SMAD7,PCYOX1L,EEF1A1.2,ACTB.2,GAPDH.1
0,56190.14,1971.443,16093.710,54993.97,40.00085,5.684647,31040.54,223.2116,-2.187667,37.01558,...,21.159680,419.1906,173.0200,111.49560,8.302612,624.8198,200.0302,39215.89,28558.14,11303.50
2,50483.09,8916.936,10451.590,48284.97,31.27827,13.904390,26012.85,431.4342,-10.483820,21.42933,...,9.122181,619.0782,139.2744,47.95465,4.253619,331.5700,160.6262,42814.16,42527.69,14241.25
4,54803.84,3906.875,11740.430,62380.23,59.56140,18.286430,28069.13,376.0385,6.185607,29.11307,...,30.748280,462.7539,180.9093,372.56140,-9.525824,307.9792,167.7340,38649.68,36737.00,11845.22
6,45847.25,6240.805,9185.637,44195.88,19.27786,44.413230,26729.13,494.9206,0.295268,25.18199,...,-4.681081,569.0219,163.2290,17.71757,-1.507952,392.2560,230.4232,39478.76,40078.56,13788.28
8,48275.86,11706.210,6891.894,50603.63,50.21552,-3.433882,23780.80,719.8223,-8.053953,15.42470,...,22.373000,629.3758,198.7061,17.82549,-7.033357,347.5759,197.3016,38290.77,49031.66,16026.82


In [5]:
group_1.head()

,EEF1A1,TUBB,TXN,ACTB,SLC35E2,PDCD1LG2,RPS28,IPO13,SYT14,AFAP1,...,SMCR7.1,NUP107,FTSJ2.1,MGC9712,TRPM3.1,SMAD7,PCYOX1L,EEF1A1.2,ACTB.2,GAPDH.1
1,52176.93,606.7773,18485.05,46674.46,35.96349,18.281510,30282.59,221.1387,0.335718,28.93709,...,13.909390,1017.3650,293.4511,51.37997,-7.676422,89.04105,297.8348,48556.75,26478.78,21009.94
3,38690.04,3148.3370,23610.13,36091.75,18.39922,1.182301,25750.38,178.7281,5.197227,24.32608,...,-3.768425,1847.4700,354.8188,67.30951,-4.145857,222.88310,215.5376,37049.47,33330.46,32106.39
5,52006.33,1553.5410,20074.07,56790.41,38.47763,3.046845,37906.21,313.6529,4.660929,24.63132,...,4.337560,1538.2070,327.6797,160.88870,3.173003,390.26060,169.4709,19928.72,35074.84,31232.45
7,66288.21,883.6413,24564.12,82740.92,37.25268,11.142720,36211.12,186.4980,3.812703,57.48767,...,9.046455,871.6389,253.3224,187.33100,8.515198,379.53390,183.5179,40934.46,27459.34,23013.48
9,44676.39,1229.1090,20303.10,42153.91,37.55210,8.733508,29171.10,239.5794,-7.967557,18.41967,...,6.796918,540.3752,242.8107,93.37427,0.251794,179.02470,326.8972,38734.47,35752.12,26704.86


In [6]:
# Calculate the mean expression for each gene in both groups
group_0_mean = group_0.mean(axis=0)  # Mean for control group
group_1_mean = group_1.mean(axis=0)  # Mean for case group

In [7]:
# Display the mean values for control and case groups
print("Normal Group Mean Expression:")
print(group_0_mean)

Normal Group Mean Expression:
EEF1A1      45792.867083
TUBB         5461.148625
TXN         11245.565958
ACTB        52727.265417
SLC35E2        27.876592
                ...     
SMAD7         435.587308
PCYOX1L       141.614702
EEF1A1.2    36510.902958
ACTB.2      41249.012500
GAPDH.1     13556.315417
Length: 22184, dtype: float64


In [8]:
print("Cancer Group Mean Expression:")
print(group_1_mean)

Cancer Group Mean Expression:
EEF1A1      44490.109167
TUBB         1146.273062
TXN         20142.365000
ACTB        42914.347917
SLC35E2        32.492770
                ...     
SMAD7         274.628823
PCYOX1L       202.447760
EEF1A1.2    36701.114167
ACTB.2      31623.007500
GAPDH.1     22382.492917
Length: 22184, dtype: float64


Compute Log2 Fold Change (log2FC)

In [9]:
# Perform t-test for each gene and calculate Log2 Fold Change
def perform_ttest_and_log2fc(group_0, group_1):
    deg_results = []
    
    for gene in group_0.columns:
        stat, p_value = ttest_ind(group_0[gene], group_1[gene], equal_var=False)

        mean_0 = np.mean(group_0[gene])
        mean_1 = np.mean(group_1[gene])
        
        # Add a small constant (1e-6) to avoid log2(0) issues
        log2fc = np.log2(mean_1 + 1e-6) - np.log2(mean_0 + 1e-6)

        deg_results.append({'Gene': gene, 'p_value': p_value, 'Log2FC': log2fc})
    
    # Convert results to DataFrame
    df_deg = pd.DataFrame(deg_results)
    
    # Multiple testing correction (FDR)
    df_deg['adjusted_p_value'] = multipletests(df_deg['p_value'], method='fdr_bh')[1]
    
    # Replace NaN values in Log2FC with 0
    df_deg['Log2FC'].fillna(0, inplace=True)

    # Filter significant DEGs based on adjusted p-value
    deg_threshold = 0.05
    df_significant = df_deg[df_deg['adjusted_p_value'] < deg_threshold].reset_index(drop=True)

    # **Apply Log2FC filtering**
    log2fc_threshold = 1  # Only consider |Log2FC| > 1
    df_filtered = df_significant[df_significant['Log2FC'].abs() > log2fc_threshold]

    return df_filtered

# Run DEG analysis
deg_filtered = perform_ttest_and_log2fc(group_0, group_1)

# Output filtered DEGs
print(f"Significant DEGs (adjusted p-value < 0.05 and |Log2FC| > 1):")
print(deg_filtered)


C:\Users\88018\AppData\Local\Temp\ipykernel_9804\3776507134.py:12: RuntimeWarning: invalid value encountered in log2
  log2fc = np.log2(mean_1 + 1e-6) - np.log2(mean_0 + 1e-6)


Significant DEGs (adjusted p-value < 0.05 and |Log2FC| > 1):
         Gene       p_value    Log2FC  adjusted_p_value
0        TUBB  4.708584e-09 -2.252254      1.206180e-07
5        CDT1  1.713530e-11  2.783929      1.532232e-09
6         LPP  4.053373e-09 -1.775566      1.073479e-07
11       SPP1  2.227755e-02  2.590832      4.976891e-02
13       MFN2  3.426842e-07 -1.434851      3.667200e-06
...       ...           ...       ...               ...
9924     SOX4  4.276329e-08  2.177517      6.844594e-07
9926  C16orf5  2.376802e-03 -1.066797      6.987851e-03
9928  SMCR7.1  2.030761e-05 -2.639152      1.107162e-04
9929   NUP107  5.246165e-10  1.341428      2.119871e-08
9930  FTSJ2.1  8.783147e-12  1.085320      9.104922e-10

[4526 rows x 4 columns]


In [10]:
# Save results
deg_filtered.to_csv("DEG_results_GSE10950.csv", index=False)

print("DEG Analysis completed! Significant genes saved in 'DEG_results_GSE10950.csv'")

DEG Analysis completed! Significant genes saved in 'DEG_results_GSE10950.csv'
